# Does Specializing More than Once Improve Learning
### Without Reg

In [5]:
from res_specialization import *
from matplotlib import pyplot as plt
import sys
import random
import pickle
import warnings

warnings.filterwarnings("ignore", category=FutureWarning) 
plt.rcParams['figure.figsize'] = [10, 5]


In [6]:
TRIALS      = 1000
NUM_TO_SPEC = 3
TOL = 5

diff_eq_params = {"x0": [-20, 10, -.5], 
                  "begin": 0, 
                  "end": 60, 
                  "timesteps":60000}

res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True,
              "solver": "least squares"
             }

results = dict()


def save_results():
    pickle.dump(results, open("spec_tw_no_reg.pkl","wb"))
# end

### Model trajectory

In [7]:
t, train_t, u = lorenz_equ(**diff_eq_params )
train_t = t[30000:55000]
test_t  = t[55000:]

### Re-run grant figure trials to get prediction length


In [8]:
random_err       = []
random_pred      = []
specialized_err  = []
specialized_pred = []


spec_sizes       = []
spec_edges       = []

for i in range(TRIALS):
    
    ######################
    # Random graph
    ######################
    
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Make predicitions
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store Error
    random_err.append(L2error(u(train_t), train_pre))
    random_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    
    #######################
    # Specialize graph
    ######################
    
    A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
    spec_sizes.append(A.shape[0])
    spec_edges.append(np.sum(A != 0))
    
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    specialized_err.append(L2error(u(train_t), train_pre))
    specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [9]:
results["random_err"]       = random_err
results["random_pred"]      = random_pred
results["specialized_err"]  = specialized_err
results["specialized_pred"] = specialized_pred
results["spec_sizes"]       = spec_sizes
results["spec_edges"]       = spec_edges
save_results()
print("Mean Prediction Length: {}".format(np.mean(random_pred)))
print("Mean Prediction Length: {}".format(np.mean(specialized_pred)))

Mean Prediction Length: 185.431
Mean Prediction Length: 2357.626


### Control: Same sizes and same edge densities as previous experiment

In [10]:
match_edge_err = []
match_edge_pred = []

for s,e in zip(spec_sizes,spec_edges):
    # Test random graph with same number of edges as specialized graphs
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    match_edge_err.append(L2error(u(train_t), train_pre))
    match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [11]:
results["match_edge_err"] = match_edge_err
results["match_edge_pred"] = match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(match_edge_pred)))
save_results()


Mean Prediction Length: 1426.943


### Specialization without finding best nodes

In [12]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True,
              "solver": "least squares"
             }

random_specialized_err  = []
random_specialized_pred = []
rspec_sizes         = []
rspec_edges         = []

for i in range(TRIALS):
    # Random graph
    rc = ResComp(3,3,**res_params)
    
    # Turn reservoir to integer adj matrix
    A = rc.res
    for j in range(A.shape[0]): A[j,j] = 0
    A = (A != 0)*1
    
    # Specialize the reservoir
    A = specializeGraph(A, random.sample(list(range(30)),27))
    
    # Store data
    rspec_sizes.append(A.shape[0])
    rspec_edges.append(np.sum(A != 0))
    
    # Make new reservoir
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Get predicitons
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store data
    random_specialized_err.append(L2error(u(train_t), train_pre))
    random_specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [13]:
results["random_specialized_err"]  = random_specialized_err
results["random_specialized_pred"] = random_specialized_pred
results["rspec_sizes"]             = rspec_sizes
results["rspec_edges"]             = rspec_edges

print("Mean Prediction Length: {}".format(np.mean(random_specialized_pred)))
save_results()


Mean Prediction Length: 2003.699


### Control: Same sizes and same edge densities as previous experiment


In [14]:
rand_match_edge_err  = []
rand_match_edge_pred = []

for s,e in zip(rspec_sizes, rspec_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    rand_match_edge_err.append(L2error(u(train_t), train_pre))
    rand_match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    

In [15]:
results["rand_match_edge_err"]  = rand_match_edge_err
results["rand_match_edge_pred"] = rand_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(rand_match_edge_pred)))
save_results()


Mean Prediction Length: 1154.797


### Targeted Specialize Twice

In [16]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True,
              "solver": "least squares"
             }

spec_tw_pre = []
spec_tw_err = []
tw_edges = []
tw_sizes = []
HOW_MANY_SPEC = 2

for i in range(TRIALS):
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 2 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        rc.fit(train_t,u)
    
    # Predict the system states
    tw_sizes.append(A.shape[0])
    tw_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_tw_err.append(L2error(u(train_t), train_pre))
    spec_tw_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))

In [17]:
results["spec_tw_pre"] = spec_tw_pre
results["spec_tw_err"] = spec_tw_err
results["tw_edges"] = tw_edges
results["tw_sizes"] = tw_sizes
save_results()


In [18]:
print("Mean Prediction Length: {}".format(np.mean(spec_tw_pre)))


Mean Prediction Length: 2648.913


### Control: Same sizes and same edge densities as previous experiment

In [19]:
tw_match_edge_err  = []
tw_match_edge_pred = []

for s,e in zip(tw_sizes, tw_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    tw_match_edge_err.append(L2error(u(train_t), train_pre))
    tw_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [20]:
results["tw_match_edge_err"] = tw_match_edge_err
results["tw_match_edge_pred"] = tw_match_edge_pred
save_results()
print("Mean Prediction Length: {}".format(np.mean(tw_match_edge_pred)))


Mean Prediction Length: 1482.698


### Targeted Specialize Three Times

In [21]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True,
              "solver": "least squares"
             }

spec_thr_pre = []
spec_thr_err = []
thr_edges = []
thr_sizes = []
HOW_MANY_SPEC = 3

for i in range(TRIALS):
    # Make RC
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 3 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        rc.fit(train_t,u)
    
    # Make predicitons
    thr_sizes.append(A.shape[0])
    thr_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_thr_err.append(L2error(u(train_t), train_pre))
    spec_thr_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))


In [22]:
results["spec_thr_pre"] = spec_thr_pre
results["spec_thr_err"] = spec_thr_err
results["thr_edges"] = thr_edges
results["thr_sizes"] = thr_sizes
print("Mean Prediction Length: {}".format(np.mean(spec_thr_pre)))

save_results()


Mean Prediction Length: 2689.933


### Control: Same sizes and same edge densities as previous experiment

In [23]:
thr_match_edge_err  = []
thr_match_edge_pred = []

for s,e in zip(thr_sizes, thr_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict systems
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    thr_match_edge_err.append(L2error(u(train_t), train_pre))
    thr_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [24]:
results["thr_match_edge_err"] = thr_match_edge_err
results["thr_match_edge_pred"] = thr_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(thr_match_edge_pred)))

save_results()


Mean Prediction Length: 1497.625


## Organize results into dataframes


In [25]:
import pandas as pd

In [26]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
df = pd.DataFrame(results)
df.describe()

,random_err,random_pred,specialized_err,specialized_pred,spec_sizes,spec_edges,match_edge_err,match_edge_pred,random_specialized_err,random_specialized_pred,...,tw_edges,tw_sizes,tw_match_edge_err,tw_match_edge_pred,spec_thr_pre,spec_thr_err,thr_edges,thr_sizes,thr_match_edge_err,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,16406.222192,185.431000,30.918552,2357.62600,116.650000,237.475000,219.811765,1426.943000,36.849514,2003.699000,...,301.612000,154.422000,126.834076,1482.698000,2689.93300,17.089228,365.672000,192.475000,62.704793,1497.625000
std,23148.510887,313.227497,181.315395,849.67689,71.636131,119.507897,906.664066,883.890839,164.402960,900.530438,...,210.674767,119.528143,475.844578,890.742063,799.51776,84.580472,222.658373,126.114735,242.359986,833.546511
min,14.783068,0.000000,7.747691,29.00000,31.000000,88.000000,5.256007,0.000000,8.708995,67.000000,...,98.000000,40.000000,5.356585,15.000000,554.00000,6.189356,110.000000,45.000000,1.916502,0.000000
25%,68.690362,36.000000,13.771475,2030.75000,74.000000,168.000000,14.199039,672.000000,15.287356,1330.000000,...,197.000000,94.000000,12.627854,694.000000,2113.00000,12.266147,228.000000,112.000000,11.326512,727.750000
50%,7368.160384,94.000000,15.712057,2145.50000,96.000000,205.000000,16.915893,1349.000000,17.292689,2054.000000,...,250.500000,124.000000,15.483280,1356.000000,2831.50000,14.086278,302.000000,157.000000,13.746926,1358.500000
75%,24028.309016,179.000000,18.035965,2873.00000,134.000000,264.000000,24.869859,2042.500000,18.664395,2793.500000,...,334.250000,176.000000,19.420736,2055.000000,2910.25000,15.897111,431.000000,229.000000,17.235569,2048.500000
max,155292.333682,2749.000000,3356.516945,5000.00000,608.000000,1069.000000,19632.548824,5000.000000,2273.258815,5000.000000,...,3571.000000,1942.000000,6668.919016,5000.000000,5000.00000,2682.567132,2289.000000,1229.000000,3586.609063,5000.000000


### Error

In [27]:
df[["random_specialized_err","specialized_err","spec_tw_err", "spec_thr_err"]].describe()

,random_specialized_err,specialized_err,spec_tw_err,spec_thr_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,36.849514,30.918552,15.353964,17.089228
std,164.402960,181.315395,8.690958,84.580472
min,8.708995,7.747691,5.966522,6.189356
25%,15.287356,13.771475,12.878485,12.266147
50%,17.292689,15.712057,14.805105,14.086278
75%,18.664395,18.035965,16.884355,15.897111
max,2273.258815,3356.516945,215.052293,2682.567132


### Error Controls

In [28]:
df[["rand_match_edge_err", "match_edge_err", "tw_match_edge_err", "thr_match_edge_err"]].describe()

,rand_match_edge_err,match_edge_err,tw_match_edge_err,thr_match_edge_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,770.449486,219.811765,126.834076,62.704793
std,2606.563412,906.664066,475.844578,242.359986
min,8.097842,5.256007,5.356585,1.916502
25%,16.521066,14.199039,12.627854,11.326512
50%,19.420946,16.915893,15.483280,13.746926
75%,402.022871,24.869859,19.420736,17.235569
max,47144.897901,19632.548824,6668.919016,3586.609063


### Prediction

In [29]:
df[["random_specialized_pred","specialized_pred","spec_tw_pre", "spec_thr_pre"]].describe()

,random_specialized_pred,specialized_pred,spec_tw_pre,spec_thr_pre
count,1000.000000,1000.00000,1000.000000,1000.00000
mean,2003.699000,2357.62600,2648.913000,2689.93300
std,900.530438,849.67689,869.999855,799.51776
min,67.000000,29.00000,426.000000,554.00000
25%,1330.000000,2030.75000,2090.750000,2113.00000
50%,2054.000000,2145.50000,2802.500000,2831.50000
75%,2793.500000,2873.00000,2910.250000,2910.25000
max,5000.000000,5000.00000,5000.000000,5000.00000


### Prediction Control

In [30]:
df[["rand_match_edge_pred", "match_edge_pred", "tw_match_edge_pred", "thr_match_edge_pred"]].describe()

,rand_match_edge_pred,match_edge_pred,tw_match_edge_pred,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,1154.797000,1426.943000,1482.698000,1497.625000
std,896.938698,883.890839,890.742063,833.546511
min,0.000000,0.000000,15.000000,0.000000
25%,403.750000,672.000000,694.000000,727.750000
50%,1217.000000,1349.000000,1356.000000,1358.500000
75%,1943.000000,2042.500000,2055.000000,2048.500000
max,5000.000000,5000.000000,5000.000000,5000.000000


### Sizes

In [31]:
sizes = df[["rspec_sizes", "spec_sizes", "tw_sizes", "thr_sizes"]]
sizes.describe()

,rspec_sizes,spec_sizes,tw_sizes,thr_sizes
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,76.766000,116.650000,154.422000,192.475000
std,38.257429,71.636131,119.528143,126.114735
min,27.000000,31.000000,40.000000,45.000000
25%,54.000000,74.000000,94.000000,112.000000
50%,66.000000,96.000000,124.000000,157.000000
75%,87.000000,134.000000,176.000000,229.000000
max,405.000000,608.000000,1942.000000,1229.000000


### Edges

In [32]:
edges = df[["rspec_edges", "spec_edges", "tw_edges", "thr_edges"]]
edges.describe()

,rspec_edges,spec_edges,tw_edges,thr_edges
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,175.986000,237.475000,301.612000,365.672000
std,65.009184,119.507897,210.674767,222.658373
min,81.000000,88.000000,98.000000,110.000000
25%,138.000000,168.000000,197.000000,228.000000
50%,159.000000,205.000000,250.500000,302.000000
75%,196.000000,264.000000,334.250000,431.000000
max,708.000000,1069.000000,3571.000000,2289.000000


### Connectivity: (num edges)/(num nodes)^2

In [33]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
edge_prob = pd.DataFrame(np.array(edges)/(np.array(sizes)**2),columns=labels)
edge_prob.describe()

,random_specialized,specialized,spec_twice,spec_three
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.037658,0.024023,0.017594,0.013877
std,0.016766,0.012503,0.009238,0.007807
min,0.004316,0.002767,0.000947,0.001515
25%,0.025499,0.014482,0.010862,0.008065
50%,0.036585,0.022400,0.016178,0.012240
75%,0.047328,0.031309,0.022654,0.018243
max,0.112483,0.091571,0.061250,0.054321


In [34]:
df[["random_err", "random_pred"]].describe()

,random_err,random_pred
count,1000.000000,1000.000000
mean,16406.222192,185.431000
std,23148.510887,313.227497
min,14.783068,0.000000
25%,68.690362,36.000000
50%,7368.160384,94.000000
75%,24028.309016,179.000000
max,155292.333682,2749.000000
